# Stats about Train / Dev / Test sets

In [1]:
import pandas as pd

import sys
sys.path.insert(0, '../')
from utils.config import PATHS
from utils.latex import show_latex, TABLES

## Load data

In [2]:
datapath = PATHS.getpath('data_expr_sept')
train = pd.read_pickle(datapath / 'clf_domains/train_excl_bck_add_pilot.pkl')
test = pd.read_pickle(datapath / 'clf_domains/test.pkl')
dev = pd.read_pickle(datapath / 'clf_domains/dev.pkl')

In [3]:
domains=['ADM', 'ATT', 'BER', 'ENR', 'ETN', 'FAC', 'INS', 'MBW', 'STM']

## Check correct split

i.e. there are no notes that appear in more than one set

In [4]:
print(test.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(train.NotitieID).any())
print(dev.NotitieID.isin(test.NotitieID).any())

False
False
False


## Number of sentences per domain

- A sentence can contain more than one domain and therefore be counted more than once.
- The last column is the total number of sentences in the dataset (incl. all negative examples)

In [5]:
caption = "Domain classification: datasets, sentence-level"
label = "domains_datasets_sents"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'pad_sen_id']]),
    columns = domains,
    data = data.labels.to_list()
)

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_sentences')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_sentences
dataset,,,,,,,,,,
dev,411,22,29,105,225,119,127,96,147,"21,742"
test,775,39,54,160,382,253,287,125,181,"22,082"
train,"4,988",247,486,989,"2,420","2,489","1,967",755,"3,390","239,153"
total,"6,174",308,569,"1,254","3,027","2,861","2,381",976,"3,718","282,977"


In [6]:
caption = "Domain classification: datasets, note-level"
label = "domains_datasets_notes"

data = pd.concat([
    train.assign(dataset = 'train'),
    test.assign(dataset = 'test'),
    dev.assign(dataset = 'dev'),
])

balance = pd.DataFrame(
    index = pd.MultiIndex.from_frame(data[['dataset', 'NotitieID']]),
    columns = domains,
    data = data.labels.to_list()
).groupby(level=[0,1]).any()

dataset_sizes = balance.pivot_table(
    index='dataset',
    aggfunc='size',
).rename('n_notes')

piv = balance.pivot_table(
    index='dataset',
    aggfunc='sum',
).join(dataset_sizes)
piv.loc['total'] = piv.sum()

piv.pipe(show_latex, caption=caption, label=label)

,ADM,ATT,BER,ENR,ETN,FAC,INS,MBW,STM,n_notes
dataset,,,,,,,,,,
dev,188,17,25,71,128,75,78,71,83,431
test,231,27,34,92,165,95,116,64,94,431
train,"2,345",175,381,707,"1,416","1,631","1,260",546,"1,989","6,821"
total,"2,764",219,440,870,"1,709","1,801","1,454",681,"2,166","7,683"


In [7]:
prefix = 'ml_datasets'
for idx, table in enumerate(TABLES):
    with open(f'./tables/{prefix}_{idx}.tex', 'w', encoding='utf8') as f:
        f.write(table)